In [32]:
import pandas as pd
import numpy as np

In [33]:
df1 = pd.read_csv("analyse_prevision/rendement.csv")
df3 = pd.read_csv("analyse_prevision/superficies.csv")

In [34]:
keep = np.unique(df3["region"])
keep = list(keep)
keep.remove("SENEGAL")
condition = [region in keep for region in df1["region"]]
df1 = df1[condition]

In [35]:
grouped = df1.groupby("indicateur")
rendements_df = grouped.get_group("Rendement kg/ha")
superficie_df = grouped.get_group("Superficie en ha")

In [36]:
superficie_df.drop(columns=["indicateur", "Unit"], inplace=True)
superficie_df.rename(columns={"Value": "superficie"}, inplace=True)
df1 = pd.merge(rendements_df, superficie_df, on=["culture", "region", "Date"])
df1

c:\Users\Massamba\anaconda3\envs\lamb_tech\lib\site-packages\pandas\core\frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\Users\Massamba\anaconda3\envs\lamb_tech\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,region,culture,indicateur,Unit,Date,Value,superficie
0,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,842.000000,13818.198922
1,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2018,805.000000,13229.000000
2,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2019,790.000000,12900.000000
3,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2020,998.000000,22738.125000
4,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2021,954.000000,11971.050000
...,...,...,...,...,...,...,...
512,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2018,3500.000000,15500.000000
513,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2019,2526.057158,10823.386290
514,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2020,2936.280998,14815.943548
515,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2021,2500.514400,12462.000000


In [37]:
factors = [0.1, 1, 0.33, 0.35, 10, 20, 30, 35, 78, 90, 100, 250, 225, 1000,1303, 1291]
new_records = pd.DataFrame(columns=df1.columns)
for factor in factors:
    new_data = df1.copy()
    new_data["Value"] /= factor
    new_data["superficie"] /= factor
    new_records = pd.concat([new_records, new_data], ignore_index=True)
new_records

,region,culture,indicateur,Unit,Date,Value,superficie
0,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,8420.000000,138181.989224
1,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2018,8050.000000,132290.000000
2,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2019,7900.000000,129000.000000
3,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2020,9980.000000,227381.250000
4,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2021,9540.000000,119710.500000
...,...,...,...,...,...,...,...
8267,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2018,2.711077,12.006197
8268,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2019,1.956667,8.383723
8269,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2020,2.274424,11.476331
8270,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2021,1.936882,9.652982


In [38]:
new_records["culture"].replace("FONIO", "OIGNON", inplace=True)
new_records.drop(columns=["indicateur","Unit"], inplace=True)
new_records

,region,culture,Date,Value,superficie
0,DIOURBEL,ARACHIDE,2017,8420.000000,138181.989224
1,DIOURBEL,ARACHIDE,2018,8050.000000,132290.000000
2,DIOURBEL,ARACHIDE,2019,7900.000000,129000.000000
3,DIOURBEL,ARACHIDE,2020,9980.000000,227381.250000
4,DIOURBEL,ARACHIDE,2021,9540.000000,119710.500000
...,...,...,...,...,...
8267,ZIGUINCHOR,RIZ,2018,2.711077,12.006197
8268,ZIGUINCHOR,RIZ,2019,1.956667,8.383723
8269,ZIGUINCHOR,RIZ,2020,2.274424,11.476331
8270,ZIGUINCHOR,RIZ,2021,1.936882,9.652982


In [39]:
grouped = new_records.groupby(["Date", "region"]).mean()

In [40]:
regions, dates, cultures, values= [], [], [], []
for region in np.unique(new_records["region"]):
    for date in np.unique(new_records["Date"]):
        regions.append(region)
        dates.append(date)
        cultures.append("PATATE")
        values.append(grouped.loc[(date, region), "Value"])

In [41]:
new_records = new_records.append(pd.DataFrame({"region": regions, "culture": cultures, "Date": dates, "Value": values}), ignore_index=True)

In [42]:
df2 = pd.read_csv("analyse_prevision/recommendation.csv")
mydict = {
    "rice": "RIZ",
    "maize": "MAIS",
    "chickpea": "NIEBE",
    "kidneybeans": "MIL",
    "pigeonpeas": "OIGNON",
    "mothbeans": "PATATE",
    "mungbean": "SORGHO",
    "blackgram": "MANIOC",
    "lentil": "ARACHIDE"
}
keep = [key for key in mydict]
condition = [label in keep for label in df2["label"]]
df2 = df2[condition]

In [43]:
def transform_label(text):
    return mydict[text]

df2["label"] = df2["label"].apply(transform_label)
df2

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,RIZ
1,85,58,41,21.770462,80.319644,7.038096,226.655537,RIZ
2,60,55,44,23.004459,82.320763,7.840207,263.964248,RIZ
3,74,35,40,26.491096,80.158363,6.980401,242.864034,RIZ
4,78,42,42,20.130175,81.604873,7.628473,262.717340,RIZ
...,...,...,...,...,...,...,...,...
895,26,56,22,23.052764,60.424786,7.011121,52.602853,ARACHIDE
896,9,77,17,21.658458,63.583371,6.280726,38.076594,ARACHIDE
897,4,59,19,26.250703,67.627797,7.621495,40.810630,ARACHIDE
898,34,73,15,20.971953,63.831799,7.630424,53.102079,ARACHIDE


In [44]:
merged_df = pd.merge(df1, df2, how='inner', left_on=['culture'], right_on=['label'])
merged_df.drop(columns=['label'], inplace=True)

In [45]:
merged_df

,region,culture,indicateur,Unit,Date,Value,superficie,N,P,K,temperature,humidity,ph,rainfall
0,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,842.000000,13818.198922,32,76,15,28.051536,63.498022,7.604110,43.357954
1,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,842.000000,13818.198922,13,61,22,19.440843,63.277715,7.728832,46.831301
2,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,842.000000,13818.198922,38,60,20,29.848231,60.638726,7.491217,46.804526
3,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,842.000000,13818.198922,11,74,17,21.363838,69.923759,6.633865,46.635286
4,DIOURBEL,ARACHIDE,Rendement kg/ha,KG/HA,2017,842.000000,13818.198922,37,71,16,26.286639,68.519667,7.324863,46.138330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49795,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2022,2662.087699,16235.100000,88,46,42,22.683191,83.463583,6.604993,194.265172
49796,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2022,2662.087699,16235.100000,93,47,37,21.533463,82.140041,6.500343,295.924880
49797,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2022,2662.087699,16235.100000,60,55,45,21.408658,83.329319,5.935745,287.576694
49798,ZIGUINCHOR,RIZ,Rendement kg/ha,KG/HA,2022,2662.087699,16235.100000,78,35,44,26.543481,84.673536,7.072656,183.622266


### Temperature dataframe

In [46]:
df3 = pd.read_csv("analyse_prevision/yield.csv")
condition = df3["annee"] >= 2008
df3 = df3[condition]

In [47]:
mysemencedict = {
    "Rice, paddy": "RIZ",
    "Maize": "MAIS",
    "Cassava": "MANIOC",
    "Potatoes": "PATATE",
    "Sorghum": "SORGHO",
    "Sweet potatoes": "OIGNON"
}

mydatedict = {
    2008: 2017,
    2009: 2018,
    2010: 2019,
    2011: 2020,
    2012: 2021,
    2013: 2022
}

def modify_date(key):
    return mydatedict[key]

def modify_culture(key):
    return mysemencedict[key]

In [48]:
df3["annee"] = df3["annee"].apply(modify_date)
df3["culture"] = df3["culture"].apply(modify_culture)
df3.rename(columns={'avg_rainfall': 'humidite'}, inplace=True)
df3.drop(columns=["id", "country", "hg/ha", "pesticides"], inplace=True)

In [49]:
meaned = df3.groupby(["annee"]).mean()
maxed = df3.groupby(["annee"]).max()
mined = df3.groupby(["annee"]).min()

In [50]:
annees, cultures, temps, humidites = [], [], [], []
for annee in np.unique(df3["annee"]): 
    # Patate
    annees.append(annee)
    cultures.append("NIEBE")
    temps.append(meaned.loc[(annee), "temp"])
    humidites.append(meaned.loc[(annee), "humidite"])
    # Mil
    annees.append(annee)
    cultures.append("MIL")
    temps.append(maxed.loc[(annee), "temp"])
    humidites.append(maxed.loc[(annee), "humidite"])
    # Arachide
    annees.append(annee)
    cultures.append("ARACHIDE")
    temps.append(maxed.loc[(annee), "temp"])
    humidites.append(maxed.loc[(annee), "humidite"])

In [51]:
df3 = df3.append(pd.DataFrame({"annee": annees, "culture": cultures, "temp": temps, "humidite": humidites}), ignore_index=True)

In [52]:
merged_df2 = pd.merge(df3, merged_df, left_on=['culture', 'annee'], right_on=['culture', 'Date'])

In [53]:
merged_df2

,culture,annee,humidite,temp,region,indicateur,Unit,Date,Value,superficie,N,P,K,temperature,humidity,ph,rainfall
0,MANIOC,2017,686.0,25.5,DIOURBEL,Rendement kg/ha,KG/HA,2017,5000.0,300.000000,56,79,15,29.484400,63.199153,7.454532,71.890907
1,MANIOC,2017,686.0,25.5,DIOURBEL,Rendement kg/ha,KG/HA,2017,5000.0,300.000000,25,62,21,26.734340,68.139997,7.040056,67.150964
2,MANIOC,2017,686.0,25.5,DIOURBEL,Rendement kg/ha,KG/HA,2017,5000.0,300.000000,42,61,22,26.272744,62.288149,7.418651,70.232076
3,MANIOC,2017,686.0,25.5,DIOURBEL,Rendement kg/ha,KG/HA,2017,5000.0,300.000000,42,73,25,34.036792,67.211138,6.501869,73.235736
4,MANIOC,2017,686.0,25.5,DIOURBEL,Rendement kg/ha,KG/HA,2017,5000.0,300.000000,44,58,18,28.036441,65.066017,6.814411,72.495077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49795,ARACHIDE,2022,686.0,24.7,ZIGUINCHOR,Rendement kg/ha,KG/HA,2022,1250.0,4806.795945,26,56,22,23.052764,60.424786,7.011121,52.602853
49796,ARACHIDE,2022,686.0,24.7,ZIGUINCHOR,Rendement kg/ha,KG/HA,2022,1250.0,4806.795945,9,77,17,21.658458,63.583371,6.280726,38.076594
49797,ARACHIDE,2022,686.0,24.7,ZIGUINCHOR,Rendement kg/ha,KG/HA,2022,1250.0,4806.795945,4,59,19,26.250703,67.627797,7.621495,40.810630
49798,ARACHIDE,2022,686.0,24.7,ZIGUINCHOR,Rendement kg/ha,KG/HA,2022,1250.0,4806.795945,34,73,15,20.971953,63.831799,7.630424,53.102079


In [54]:
winds = {
    2017: 12,
    2018: 10,
    2019: 8,
    2020: 13,
    2021: 10,
    2022: 9
}

merged_df2["wind"] = [winds[year] for year in merged_df2["annee"]]
merged_df2.drop(columns=["Date", "Unit", "indicateur", "annee"], inplace=True)
merged_df2

,culture,humidite,temp,region,Value,superficie,N,P,K,temperature,humidity,ph,rainfall,wind
0,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,56,79,15,29.484400,63.199153,7.454532,71.890907,12
1,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,25,62,21,26.734340,68.139997,7.040056,67.150964,12
2,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,42,61,22,26.272744,62.288149,7.418651,70.232076,12
3,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,42,73,25,34.036792,67.211138,6.501869,73.235736,12
4,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,44,58,18,28.036441,65.066017,6.814411,72.495077,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49795,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,26,56,22,23.052764,60.424786,7.011121,52.602853,9
49796,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,9,77,17,21.658458,63.583371,6.280726,38.076594,9
49797,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,4,59,19,26.250703,67.627797,7.621495,40.810630,9
49798,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,34,73,15,20.971953,63.831799,7.630424,53.102079,9


In [55]:
merged_df2.dropna(inplace=True)
merged_df2

,culture,humidite,temp,region,Value,superficie,N,P,K,temperature,humidity,ph,rainfall,wind
0,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,56,79,15,29.484400,63.199153,7.454532,71.890907,12
1,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,25,62,21,26.734340,68.139997,7.040056,67.150964,12
2,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,42,61,22,26.272744,62.288149,7.418651,70.232076,12
3,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,42,73,25,34.036792,67.211138,6.501869,73.235736,12
4,MANIOC,686.0,25.5,DIOURBEL,5000.0,300.000000,44,58,18,28.036441,65.066017,6.814411,72.495077,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49795,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,26,56,22,23.052764,60.424786,7.011121,52.602853,9
49796,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,9,77,17,21.658458,63.583371,6.280726,38.076594,9
49797,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,4,59,19,26.250703,67.627797,7.621495,40.810630,9
49798,ARACHIDE,686.0,24.7,ZIGUINCHOR,1250.0,4806.795945,34,73,15,20.971953,63.831799,7.630424,53.102079,9


In [56]:
merged_df2.drop(columns=["temp","humidite"], inplace=True)
non_customized_df = merged_df2.drop(columns=["N","P", "K", "ph"])

### Model Pipeline

In [57]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X = merged_df2.drop(columns=['Value'])
y = merged_df2['Value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

one_hot_cols = ['culture', 'region']
scale_cols = ['superficie', 'N', 'P', 'K', 'temperature', 'humidity', 'ph', 'rainfall', 'wind']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), one_hot_cols),
        ('scaler', MinMaxScaler(), scale_cols)
    ]
)

models = [
    ('GradientBoostingRegressor', GradientBoostingRegressor())
]

param_grids = [
    {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.05, 0.1, 0.2]
    }
]

results = []

for (name, model), param_grid in zip(models, param_grids):
    print(name)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X, y)
    results.append((name, grid_search.best_score_, grid_search.best_params_))

best_model_name, best_score, best_params = max(results, key=lambda x: x[1])
best_model = models[[name for name, _, _ in results].index(best_model_name)][1]

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])

print("Best Model:", best_model_name)
print("Best Score:", best_score)
print("Best Parameters:", best_params)

GradientBoostingRegressor
Best Model: GradientBoostingRegressor
Best Score: -2213150.538676883
Best Parameters: {'model__learning_rate': 0.2, 'model__n_estimators': 50}


In [58]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

final_pipeline.fit(X, y)
y_pred = final_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 417485.8774844114
Root Mean Squared Error (RMSE): 646.1314707429219
Mean Absolute Error (MAE): 418.0206844580421
R-squared (R2) Score: 0.9754716139637498


In [59]:
import joblib

joblib.dump(final_pipeline, 'models/final_pipeline.joblib')
print("Final pipeline saved successfully.")

Final pipeline saved successfully.


### Model Pipeline for non custom

In [60]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X = merged_df2.drop(columns=['Value'])
y = merged_df2['Value']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

one_hot_cols = ['culture', 'region']
scale_cols = ['superficie','temperature', 'humidity','rainfall', 'wind']

preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', OneHotEncoder(), one_hot_cols),
        ('scaler', MinMaxScaler(), scale_cols)
    ]
)

models = [
    ('GradientBoostingRegressor', GradientBoostingRegressor())
]

param_grids = [
    {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.05, 0.1, 0.2]
    }
]

results = []

for (name, model), param_grid in zip(models, param_grids):
    print(name)
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X, y)
    results.append((name, grid_search.best_score_, grid_search.best_params_))

best_model_name, best_score, best_params = max(results, key=lambda x: x[1])
best_model = models[[name for name, _, _ in results].index(best_model_name)][1]

final_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', best_model)
])

print("Best Model:", best_model_name)
print("Best Score:", best_score)
print("Best Parameters:", best_params)

GradientBoostingRegressor
Best Model: GradientBoostingRegressor
Best Score: -2197107.9649912566
Best Parameters: {'model__learning_rate': 0.2, 'model__n_estimators': 50}


In [61]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

final_pipeline.fit(X, y)
y_pred = final_pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)

Mean Squared Error (MSE): 430770.97545456415
Root Mean Squared Error (RMSE): 656.3314524343353
Mean Absolute Error (MAE): 421.1591400588243
R-squared (R2) Score: 0.9744191386480188


In [62]:
import joblib

joblib.dump(final_pipeline, 'models/non_custom_final_pipeline.joblib')
print("Final pipeline saved successfully.")

Final pipeline saved successfully.
